In [ ]:
from utils.CustomDataset import Fets2022Noise, Fets2022
import torch
import SimpleITK as sitk
from Network.Unet import Custom3DUnet
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import SimpleITK as sitk
from skimage import measure
import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
import random

index = 36
depth = 76
skip = False
nslice= 15
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def convert_to_one_hot(label, classes = 4):
    return torch.nn.functional.one_hot(torch.Tensor(label).type(torch.int64), classes).detach().numpy()
def display(orient, label, num_classes=4):

    fig, ax=plt.subplots(1, 3)

    ax[0].imshow(label, cmap="magma")
    ax[1].imshow(orient, cmap="gray")
    ax[2].imshow(orient, 'gray')
    for indx in range(4):
        if indx == 0:
            continue
        countour = measure.find_contours(convert_to_one_hot(label, num_classes)[...,indx], 0.7)
        for legion in countour:
            
            y=legion.T[0].copy()
            x=legion.T[1].copy()
            legion[:,0]= x
            legion[:,1]= y
            poly=Polygon(legion, closed=False)
            if indx == 1:
                poly.set_color([0,1,1,0.3])
            if indx == 2:
                poly.set_color([1,0,0,0.3])
            if indx == 3:
                poly.set_color([1,0.95,0,0.3])
            ax[2].add_patch(poly)
    ax[0].set_axis_off()
    ax[1].set_axis_off()
    ax[2].set_axis_off()
    fig.tight_layout()
    fig.show()

In [ ]:
diceLoss = smp.losses.DiceLoss(
   mode="multiclass",          # For multi-class segmentation
   classes=None,               # Compute the loss for all classes
   log_loss=False,             # Do not use log version of Dice loss
   from_logits=True,           # Model outputs are raw logits
   smooth=1e-5,                # A small smoothing factor for stability
   ignore_index=None,          # Don't ignore any classes
   eps=1e-7                    # Epsilon for numerical stability
)
class Hausdolf95(torch.nn.Module):
    def __init__(self):
        super(Hausdolf95, self).__init__()
        self.distancef = torch.nn.PairwiseDistance()
    def distance(self, x, y):
        return self.distancef(x, y)
        
    def forward(self, x, y):
        maximum_d=torch.max(self.distance(x, y))
        return maximum_d
hausdolf95 = Hausdolf95()
jaccardf = smp.losses.JaccardLoss(
   mode="multiclass",          # For multi-class segmentation
   classes=None,               # Compute the loss for all classes
   log_loss=False,             # Do not use log version of Dice loss
   from_logits=True,           # Model outputs are raw logits
   smooth=1e-5,                # A small smoothing factor for stability          
   eps=1e-7
)
def evaluation(net, loader, diceLoss, HF95, jaccard,DEVICE):
    net.eval()
    net.to(DEVICE)
    dicef= diceLoss.to(DEVICE)
    hf95f = HF95.to(DEVICE)
    jaccardf = jaccard.to(DEVICE)
    dice = 0
    hf95 = 0
    jacc = 0
    length = len(loader)
    for sample in loader:
        X= torch.stack([s["x"] for s in sample], 0)
        Y= torch.stack([s["y"] for s in sample], 0)
        out = net(X.type(torch.float32).unsqueeze(0).to(DEVICE)) 
        dice += (1-dicef(out.type(torch.float32).to(DEVICE), Y.type(torch.int64).to(DEVICE))).item()
        jacc += (1-jaccardf(out.type(torch.float32).to(DEVICE), Y.type(torch.int64).to(DEVICE))).item()
        hf95 += hf95f(out.squeeze().type(torch.float32).to(DEVICE), torch.nn.functional.one_hot(Y.type(torch.int64).squeeze(), 4).permute(3, 0, 1, 2).type(torch.float32).to(DEVICE)).item()
    return {"dice": dice/length, "hf95":hf95/length, 'jaccard': jacc/length}

def visualization(net, fets2023, index, depth, DEVICE):
    net.to(DEVICE)
    net.eval()
    ori = sitk.GetImageFromArray(fets2023[index]['x'].detach().numpy()[depth, ...], False)
    lab = sitk.GetImageFromArray(fets2023[index]['y'].type(torch.int32).detach().numpy()[depth ,...], False)
    label = sitk.GetArrayFromImage(lab)
    orient = sitk.GetArrayFromImage(ori)
    out = net(fets2023[index]['x'].unsqueeze(0).unsqueeze(0).to(DEVICE))
    display(orient, label)
    display(orient, out.cpu().squeeze().argmax(0).detach().numpy()[depth])

def visualization2(net, fets2023, index, nslice,DEVICE):
    net.to(DEVICE)
    net.eval()
    ori = sitk.GetImageFromArray(fets2023[index]['x'].detach().numpy(), False)
    lab = sitk.GetImageFromArray(fets2023[index]['y'].type(torch.int32).detach().numpy(), False)
    label = sitk.GetArrayFromImage(lab)
    orient = sitk.GetArrayFromImage(ori)
    out = net(fets2023[index]['x'].unsqueeze(0).unsqueeze(0).to(DEVICE))
    display2(orient.squeeze(), label.squeeze())
    display2(orient.squeeze(), out.cpu().squeeze().argmax(0).detach().numpy())

def display2(orient, label, nslice=15, num_classes=4):
    depth = orient.shape[0]
    ranger = range(nslice, nslice*(depth//nslice), nslice)
    fig, ax=plt.subplots(1, len(ranger))
    fig.set_size_inches(16, 16)
    for axind,i in enumerate(range(nslice, nslice*(depth//nslice), nslice)):
        ax[axind].imshow(orient[i,...], cmap="gray")
    
        for indx in range(4):
            if indx == 0:
                continue
            countour = measure.find_contours(convert_to_one_hot(label[i,...], num_classes)[...,indx], 0.7)
            for legion in countour:
                
                y=legion.T[0].copy()
                x=legion.T[1].copy()
                legion[:,0]= x
                legion[:,1]= y
                poly=Polygon(legion, closed=False)
                if indx == 1:
                    poly.set_color([0,1,1,0.6])
                if indx == 2:
                    poly.set_color([1,0,0,0.6])
                if indx == 3:
                    poly.set_color([0.85,0.75,0,0.6])
                ax[axind].add_patch(poly)
        ax[axind].set_axis_off()
    fig.tight_layout()
    fig.show()

In [ ]:
fets2023 = Fets2022("Data/Fets2022/Partitioning1/test1", 5)
visualset = Fets2022("Data/Fets2022/Partitioning1/client1", 5)
fetsloader = DataLoader(fets2023, 1, False, collate_fn=lambda x:x)
net = Custom3DUnet(1, 4, True, f_maps=4, layer_order="gcr", num_groups=4)
net.load_state_dict(torch.load("./Models/CentralFeTs2021/net.pt"))
visualization(net, fets2023, index, depth, DEVICE)

In [ ]:
net = Custom3DUnet(1, 4, True, f_maps=4, layer_order="gcr", num_groups=4)
net.load_state_dict(torch.load("./Models/CentralFeTs2021/net.pt"))
visualization2(net, visualset, index, nslice, DEVICE)
if not skip:
    print(evaluation(net, fetsloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("./Models/FedAvgFeTs2021/net.pt"))
visualization2(net, visualset, index, nslice, DEVICE)
if not skip:
    print(evaluation(net, fetsloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("./Models/FedPIDFeTs2021/net.pt"))
visualization2(net, visualset, index, nslice, DEVICE)
if not skip:
    print(evaluation(net, fetsloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("./Models/FedLWRFeTs2021/net.pt"))
visualization2(net, visualset, index, nslice, DEVICE)
if not skip:
    print(evaluation(net, fetsloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
net.load_state_dict(torch.load("./Models/FedRefFeTs2021/net.pt"))
visualization2(net, visualset, index, nslice, DEVICE)
if not skip:
    print(evaluation(net, fetsloader, diceLoss, hausdolf95, jaccardf, DEVICE))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
fedavgframe = pd.read_csv('Result/fedavg/FedAvg_fets.csv')
fedpidframe = pd.read_csv('Result/fedpid/FedPID_fets.csv')
fedlwrframe = pd.read_csv('Result/fedlwr/FedLWR_fets.csv')
fedrefframe = pd.read_csv("Result/fedref/FedRef_fets.csv")

length = min([len(fedavgframe), len(fedpidframe), len(fedlwrframe), len(fedrefframe)])
plt.plot(fedavgframe['loss'].to_numpy()[:length], color=(0.5,0,1), label= "Fed-Avg", marker= ".", linestyle ="--")
plt.plot(fedlwrframe['loss'].to_numpy()[:length], color=(0.5,1,0.9), label= "Fed-LWR", marker= ".", linestyle ="--")
plt.plot(fedpidframe['loss'].to_numpy()[:length], color=(0.5,1,0), label= "Fed-PID", marker= ".", linestyle ="--")
plt.plot(fedrefframe['loss'].to_numpy()[:length], color=(1,0,0), label= "Fed-Ref", marker= "v", linestyle ="--")
plt.grid(True, axis="y", alpha=0.5, linestyle="--")
plt.legend(fontsize=16)
plt.xlabel("round".upper())
# plt.ylabel("Loss")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
fedavgframe = pd.read_csv('Result/fedavg/FedAvg_fets.csv')
fedpidframe = pd.read_csv('Result/fedpid/FedPID_fets.csv')
fedlwrframe = pd.read_csv('Result/fedlwr/FedLWR_fets.csv')
fedrefframe = pd.read_csv("Result/fedref/FedRef_fets.csv")
length = min([len(fedavgframe), len(fedpidframe), len(fedlwrframe), len(fedrefframe)])
plt.plot(fedavgframe['mDice'].to_numpy()[:length], color=(0.5,0,1), label= "Fed-Avg", marker= ".", linestyle ="--")
plt.plot(fedlwrframe['mDice'].to_numpy()[:length], color=(0.5,1,0.9), label= "Fed-LWR", marker= ".", linestyle ="--")
plt.plot(fedpidframe['mDice'].to_numpy()[:length], color=(0.5,1,0), label= "Fed-PID", marker= ".", linestyle ="--")
plt.plot(fedrefframe['mDice'].to_numpy()[:length], color=(1,0,0), label= "Fed-Ref", marker= "v", linestyle ="--")
plt.grid(True, axis="y", alpha=0.5, linestyle="--")
plt.legend(fontsize=16)
plt.xlabel("round".upper())
# plt.ylabel("DiceScore")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
 
fedavgframe = pd.read_csv('Result/fedavg/FedAvg_fets.csv')
fedpidframe = pd.read_csv('Result/fedpid/FedPID_fets.csv')
fedlwrframe = pd.read_csv('Result/fedlwr/FedLWR_fets.csv')
fedrefframe = pd.read_csv("Result/fedref/FedRef_fets.csv")
length = min([len(fedavgframe), len(fedpidframe), len(fedlwrframe), len(fedrefframe)])
plt.plot(fedavgframe['mHF95'].to_numpy()[:length], color=(0.5,0,1), label= "Fed-Avg", marker= ".", linestyle ="--")
plt.plot(fedlwrframe['mHF95'].to_numpy()[:length], color=(0.5,1,0.9), label= "Fed-LWR", marker= ".", linestyle ="--")
plt.plot(fedpidframe['mHF95'].to_numpy()[:length], color=(0.5,1,0), label= "Fed-PID", marker= ".", linestyle ="--")
plt.plot(fedrefframe['mHF95'].to_numpy()[:length], color=(1,0,0), label= "Fed-Ref", marker= "v", linestyle ="--")
plt.grid(True, axis="y", alpha=0.5, linestyle="--")
plt.legend(fontsize=16)
plt.xlabel("round".upper())
# plt.ylabel("Housdorff Distance")

In [ ]:
import pandas as pd

df = pd.read_csv('Data/Fets2022/MICCAI_FeTS2021_TrainingData/partitioning_1.csv')
c1 = df[df['Partition_ID']==1]["Subject_ID"]
c2 = df[df['Partition_ID']==2]["Subject_ID"]
c3 = df[df['Partition_ID']==3]["Subject_ID"]
c4 = df[df['Partition_ID']==4]["Subject_ID"]
c5 = df[df['Partition_ID']==5]["Subject_ID"]
c6 = df[df['Partition_ID']==6]["Subject_ID"]
c7 = df[df['Partition_ID']==7]["Subject_ID"]
c8 = df[df['Partition_ID']==8]["Subject_ID"]
c9 = df[df['Partition_ID']==9]["Subject_ID"]
c10 = df[df['Partition_ID']==10]["Subject_ID"]
c11 = df[df['Partition_ID']==11]["Subject_ID"]
c12 = df[df['Partition_ID']==12]["Subject_ID"]
c13 = df[df['Partition_ID']==13]["Subject_ID"]
c14 = df[df['Partition_ID']==14]["Subject_ID"]
c15 = df[df['Partition_ID']==15]["Subject_ID"]
c16 = df[df['Partition_ID']==16]["Subject_ID"]
c17 = df[df['Partition_ID']==17]["Subject_ID"]

In [ ]:
import matplotlib.pyplot as plt

plt.bar(range(1, 18, 1), sorted([len(locals()[f"c{i}"]) for i in range(1, 18)], reverse=True))
plt.xlim(0.5, 17.5)
plt.xlabel("Client ID")
plt.ylabel("Volumes")
plt.title("partitioning 1")

In [ ]:
df = pd.read_csv('Data/Fets2022/MICCAI_FeTS2021_TrainingData/partitioning_2.csv')
c1 = df[df['Partition_ID']==1]["Subject_ID"]
c2 = df[df['Partition_ID']==2]["Subject_ID"]
c3 = df[df['Partition_ID']==3]["Subject_ID"]
c4 = df[df['Partition_ID']==4]["Subject_ID"]
c5 = df[df['Partition_ID']==5]["Subject_ID"]
c6 = df[df['Partition_ID']==6]["Subject_ID"]
c7 = df[df['Partition_ID']==7]["Subject_ID"]
c8 = df[df['Partition_ID']==8]["Subject_ID"]
c9 = df[df['Partition_ID']==9]["Subject_ID"]
c10 = df[df['Partition_ID']==10]["Subject_ID"]
c11 = df[df['Partition_ID']==11]["Subject_ID"]
c12 = df[df['Partition_ID']==12]["Subject_ID"]
c13 = df[df['Partition_ID']==13]["Subject_ID"]
c14 = df[df['Partition_ID']==14]["Subject_ID"]
c15 = df[df['Partition_ID']==15]["Subject_ID"]
c16 = df[df['Partition_ID']==16]["Subject_ID"]
c17 = df[df['Partition_ID']==17]["Subject_ID"]
import matplotlib.pyplot as plt

plt.bar(range(1, 18, 1), sorted([len(locals()[f"c{i}"]) for i in range(1, 18)], reverse=True))
plt.xlim(0.5, 17.5)
plt.xlabel("Client ID")
plt.ylabel("Volumes")
plt.title("partitioning 2")